### Importing Libraries
<li> <code>requests</code>: Used to send HTTP requests to a URL.</li>
<li> <code>BeautifulSoup</code>: Utilized for parsing HTML documents and extracting data.</li>
<li> <code>pandas</code>: Offers data structures and data analysis tools, here specifically used for creating and manipulating a DataFrame.</li>
<li> <code>numpy</code> and <code>datetime</code>: Imported for potential future steps in the ETL process, such as data transformation involving dates or numerical operations.</li>
<li> <code>psycopg2</code>: A PostgreSQL adapter for Python, likely intended for use in the Load phase to insert data into a PostgreSQL database.</li>

In [2]:
# Importing necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import psycopg2 

### Setting up Initial Variables
<li> <code>url</code>: Specifies the webpage containing the GDP data to be scraped.</li>
<li> <code>table_attribs</code>: Defines the column names for the DataFrame that will store the extracted data.</li>

In [3]:
# The URL of the webpage to scrape
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
# A list of column names for the DataFrame
table_attribs = ["Country","Region","GDP_USD_Millions", "Year"]

db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'

### Data Extraction

##### Defining the <code>extract</code> function
This function aims to scrape the specified URL, extract data according to the structure of the HTML,<br>and organize this data into a pandas DataFrame with predefined columns (<b>Country</b>, <b>Region</b>, and <b>GDP_USD_Millions</b>).

##### Fetching and Parsing the Webpage
<li> Sends an HTTP GET request to the specified URL and fetches the HTML content of the page.</li>
<li> Parses the HTML content, creating a <code>BeautifulSoup</code> object for easy navigation and search within the HTML document.</li>

##### Extracting Target Table and Processing each row

<li>Finds all <code>tbody</code> elements (which enclose table rows in HTML) to locate tables within the page. The third table (<code>tables[2]</code>, <br>as indexing starts at 0) is assumed to contain the desired data.</li>
<li>Retrieves all row elements (<code>tr</code>) from the selected table body for processing.</li>
<li>Iterates over each row, extracting data cells (<code>td</code> tags) within.</li>
<li>Ensures the row contains data cells before proceeding.</li>


##### Extracting and Cleaning, Validating and Appending Data
<li>For each data cell, the country name and region are extracted and cleaned of any surrounding whitespace.</li>
<li>The GDP value is also extracted and cleaned of commas and dollar signs to facilitate later conversion to a numeric type.</li>
<li>Checks for the presence of a placeholder character ('—') to ensure the GDP data is valid.</li>
<li>Constructs a dictionary from the extracted data and converts it into a DataFrame row.</li>
<li>Concatenates this row with the existing DataFrame, efficiently accumulating the extracted data.</li>

In [4]:
# Code for ETL operations on Country-GDP data
def extract(url, table_attribs):
    ''' 
        This function extracts the required information from the website and saves it to a dataframe. 
        The function returns the dataframe containing the extracting data. 
    '''
    # Fetch the HTML content of the page
    page = requests.get(url).text
    # Parse the HTML content using BeautifulSoup
    data = BeautifulSoup(page, 'html.parser')
    
    # Initialize an empty DataFrame with specified column names
    df = pd.DataFrame(columns=table_attribs)
    
    # Find all 'tbody' tags to locate tables in the page
    tables = data.find_all('tbody')
    # Select the 3rd table and find all its row elements ('tr')
    rows = tables[2].find_all('tr')
    
    # Iterate through each row in the table
    for row in rows:
        # Find all data cells (columns) in the row ('td' tags)
        col = row.find_all('td')
        
        # Proceed if there are more than one column, ensuring it's a data row
        if len(col) > 0:
            # Extract the country and region, retrieving all text in the cell including 'a' tags
            # Removes links and leading / trailing whitespaces
            country = col[0].get_text().strip()
            region = col[1].get_text().strip()
            # Extract the GDP value, removing any commas and dollar signs for easier numeric conversion
            gdp = col[2].text.replace(',','').replace('$', '')
            year = col[3].get_text().strip()
            
            # Ensure the GDP data is valid, and is not filled the GDP values (not filled = with text like '—')
            if '—' not in gdp:
                # Create a dictionary with the extracted data, using predefined keys
                data_dict = {"Country": country, "Region": region, "GDP_USD_Millions": gdp, "Year": year}
                # Convert dictionary into DataFrame
                # Concatenate it with the existing DataFrame
                df = pd.concat([df, pd.DataFrame([data_dict])], ignore_index = True)
    return df

df = extract(url, table_attribs)
df

,Country,Region,GDP_USD_Millions,Year
0,World,—,105568776,2023
1,United States,Americas,26854599,2023
2,China,Asia,19373586,[n 1]2023
3,Japan,Asia,4409738,2023
4,Germany,Europe,4308854,2023
...,...,...,...,...
187,Marshall Islands,Oceania,291,2023
188,Palau,Oceania,262,2023
189,Kiribati,Oceania,248,2023
190,Nauru,Oceania,151,2023


### Data Transformation
In the Transformation phase of our ETL process, we aim to refine the GDP data by ensuring it's in the correct numerical format, adjusting the scale from<br> millions to billions, and renaming the column to reflect this change. Also we add additional the YEAR column as additional information

##### GDP Data Transformation
<li><b>Ensure GDP is in Floating Point Format</b>: First, we need to ensure that the <code>GDP_USD_Millions</code> column is in a floating-point number format<br> to perform mathematical operations. This is important because the data extracted from HTML pages often comes in as strings.</li>
<li><b>Convert Millions to Billions</b>: Next, we'll convert the GDP values from millions of USD to billions of USD. This involves dividing each value by 1000<br> and rounding the result to two decimal places.</li>
<li><b>Rename the Column</b>: Finally, we'll rename the column from <code>GDP_USD_Millions</code> to <code>GDP_USD_Billions</code>. This line changes the column name in the<br> DataFrame <code>df</code>, with <code>inplace=True</code> indicating that the DataFrame is modified in place (no need to assign the result to a new DataFrame).</li>

##### Year Data Transformation
<li><b>String Conversion and Formatting</b>: The function initially treats each value in the <code>"Year"</code> column as a string, regardless of its original type (e.g., int, <br>str, or Timestamp). It does this to standardize the processing.
<li><b>Year Extraction and Date Format</b>: It then extracts the last four characters of this string, assuming they represent a year (e.g., "2023" from "[n 1]2023"). <br>If these characters are digits <code>(.isdigit() checks this)</code>, it appends <code>"-12-31"</code> to create a string representing December 31 of that year. This process <br>transforms a year into a specific date, marking the end of that year.
<li><b>Handling Non-Year Strings</b>: If the last four characters aren't digits, indicating that a valid year couldn't be extracted, it defaults to <code>"1900-12-31"</code>. <br>This ensures all rows have a consistent format.
<li><b>Datetime Conversion</b>: Next, it converts these strings into pandas datetime objects <code>(pd.to_datetime)</code>, facilitating any future date-related operations.

In [5]:
def transform(df):
    ''' 
        Transforms the GDP data from millions to billions with decimanls, and renames the column
        Returns the DataFrame with GDP data transformed to billions
    '''
    # Ensure Year is treated as a string, applying transformation only if it's digit (to handle blanks)
    df['Year'] = df['Year'].apply(lambda x: f'{str(x)[-4:]}-12-31' if str(x)[-4:].isdigit() else '1900-12-31')
    # Transform the Year column to datetime format
    df['Year'] = pd.to_datetime(df['Year'], format='%Y-%m-%d')
    # Transform the Region column to replace with nulls when not found region
    df['Region'] = df['Region'].replace({'—': ''})
    
    # Ensure GDO values are in floating point format
    df['GDP_USD_Millions'] = df['GDP_USD_Millions'].astype(float)
    # Convert GDP from millions to billions and round to 2 decimal places
    df['GDP_USD_Millions'] = df['GDP_USD_Millions'] / 1000
    df['GDP_USD_Millions'] = df['GDP_USD_Millions'].round(2)
    # Rename the column to reflect the new unit of measure
    df.rename(columns={'GDP_USD_Millions' : 'GDP_USD_Billions'}, inplace=True)
    df.rename(columns={'Year': 'Period'}, inplace=True)
    
    return df
transformed_df = transform(df)


In [6]:
transformed_df

,Country,Region,GDP_USD_Billions,Period
0,World,,105568.78,2023-12-31
1,United States,Americas,26854.60,2023-12-31
2,China,Asia,19373.59,2023-12-31
3,Japan,Asia,4409.74,2023-12-31
4,Germany,Europe,4308.85,2023-12-31
...,...,...,...,...
187,Marshall Islands,Oceania,0.29,2023-12-31
188,Palau,Oceania,0.26,2023-12-31
189,Kiribati,Oceania,0.25,2023-12-31
190,Nauru,Oceania,0.15,2023-12-31


In [12]:
import psycopg2
import json
import pandas as pd
import uuid

import psycopg2.extras

def load_db_config(config_path = 'Config DB/config.json'):
    with open(config_path, 'r') as config_file:
        return json.load(config_file)

def load_data_to_db(transformed_df):
    try:
        # Connect to the data base
        db_params = load_db_config()
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()
        print("Connection established.")
        
        # Generate a unique batch ID for this execution
        batch_id = str(uuid.uuid4())
        
        # Iterate through DataFrame rows
        for index, row in transformed_df.iterrows():
            # Prepare data for the stored procedure call
            country = str(row['Country'])
            region = str(row['Region'])
            gdp = float(row['GDP_USD_Billions'])
            period = pd.to_datetime(row['Period']).strftime('%Y-%m-%d')
            
            # Call the stored procedure for each row
            cursor.execute("CALL etl.update_gdp_data(%s, %s, %s, %s, %s)", 
                           (country, region, gdp, period, batch_id))
        # Commit the transactions after all rows have been processed
        conn.commit()
        
        # Retrieve and display the log entries for this batch
        cursor.execute( "SELECT action_taken, count(*) FROM etl.countries_gdp_update_log WHERE batch_id = %s GROUP BY action_taken;", (batch_id,))
        action_counts = cursor.fetchall()
        
        if action_counts:
            action_summary = ', '.join([f"{action} = {count}" for action, count in action_counts])
        else:
            action_summary = "No Records"
        
        return True, f"Operation completed. \n Records Inserted / Updated: {action_summary} \n Batch ID: {batch_id}."
        
        
    except Exception as e:
        if conn:
            conn.rollback() # Roll back the transaction on error
        return False, f"SQL error: {type(e).__name__}, {e}"
    
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
    

success, message = load_data_to_db(transformed_df)
print(message)

Connection established.
Operation completed. 
 Records Inserted / Updated: No Records 
 Batch ID: d206dc98-a3b6-413c-a949-96521f918b9f.
